In [2]:
import pandas as pd
import re
import datetime
import numpy as np
import matplotlib.pyplot as plt
import string 
import warnings
warnings.simplefilter('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import recall_score, roc_auc_score, accuracy_score, plot_confusion_matrix, classification_report
#from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from regressors import stats
from sklearn.feature_selection import chi2

In [3]:
train_original = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/02_eda_data/output/train_dataset.csv")
test_original = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/02_eda_data/output/test_dataset.csv")
resume_original = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/06_clean_data/05182020_cleaned_english_resumes_V1.0.csv")
avg_work = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/06_clean_data/avg_work_exp.csv")
competitor = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/06_clean_data/competitor_experience.csv")
education_concentration = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/06_clean_data/education_concentration.csv")
education_concentration.rename(columns = {"interactive arts and technology":"interactive_arts_and_technology_concentration"},inplace = True)
work_title = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/06_clean_data/work_title.csv")
degree = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/06_clean_data/features_edu_01_training_df.csv")
sales_cus_exp = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/06_clean_data/sales_custom_exp.csv")
#word_counts = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/06_clean_data/word_counts.csv")
consolidate_dataset = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/06_clean_data/consolidated_features_01_training_dataset.csv")
communication_level = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/06_clean_data/communication_level.csv")
industry_exp = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/06_clean_data/industry_exp.csv")

In [4]:
list(education_concentration.columns)

['Unnamed: 0',
 'employee_code',
 'employee_name',
 'education1_label',
 'education2_label',
 'education3_label',
 'accounting_concentration',
 'arts_concentration',
 'audio_technician_concentration',
 'business_concentration',
 'communication_concentration',
 'computer_systems_concentration',
 'criminology_concentration',
 'dental_concentration',
 'economics_concentration',
 'education_concentration',
 'engineering_concentration',
 'english_concentration',
 'finance_concentration',
 'general_concentration',
 'healthcare_concentration',
 'hospitality_concentration',
 'human_resource_concentration',
 'interactive_arts_and_technology_concentration',
 'kinesiology_concentration',
 'law_concentration',
 'marketing_concentration',
 'not_specified_concentration',
 'other_concentration',
 'physics_concentration',
 'psychology_concentration',
 'science_concentration',
 'sociology_concentration',
 'statistics_concentration']

In [5]:
consolidate_dataset.columns

Index(['Unnamed: 0', 'employee_code', 'clean_text', 'gender', 'generation',
       'hp_class', 'perf_found', 'rehired_', 'referral_flag', 'exclusion_code',
       'highest_degree', 'background_highest_degree', 'country_highest_degree',
       'no_lang_spoken', 'trilingual_flag', 'goal_record',
       'sales_customer_base_exp', 'volunteer_exp', 'problem_solver',
       'sports_mention', 'communication_skills', 'team_player',
       'leadership_mention', 'sales_exp_months', 'customer_serv_exp_months',
       'leader_ship_exp_months', 'average_tenure_per_job', 'job_hopper',
       'total_experience_months', 'longest_tenure', 'shortest_tenure',
       'food_service_industry_exp', 'apparel_industry_exp',
       'supercenter_convenience_industry_exp', 'automotive_sales_industry_exp',
       'blue_collar_industry_exp', 'consumer_electronics', 'age'],
      dtype='object')

In [33]:
education_concentration.drop_duplicates("employee_code", inplace=True)
work_title.drop_duplicates("employee_code", inplace=True)
degree.drop_duplicates("employee_code", inplace=True)
sales_cus_exp.drop_duplicates("employee_code", inplace=True)
consolidate_dataset.drop_duplicates("employee_code", inplace=True)
communication_level.drop_duplicates("employee_code", inplace=True)
industry_exp.drop_duplicates("employee_code", inplace=True)
consolidate_dataset = consolidate_dataset[['employee_code','no_lang_spoken', 'trilingual_flag', 'goal_record',\
               'sales_customer_base_exp', 'volunteer_exp', 'problem_solver',\
               'sports_mention', 'communication_skills', 'team_player',\
               'leadership_mention','average_tenure_per_job','shortest_tenure',]]

In [34]:
train = pd.merge(train_original, resume_original, how="left", on="employee_code").merge(
                                     avg_work, how="left", on="employee_code").merge(
                                     competitor, how="left", on="employee_code").merge(
                                    education_concentration, how="left", on="employee_code").merge(
                                    work_title, how = "left", on="employee_code").merge(
                                    degree, how="left", on="employee_code").merge(
                                    sales_cus_exp, how="left", on="employee_code").merge(
                                    communication_level, how="left", on="employee_code").merge(
                                    industry_exp, how="left", on="employee_code").merge(
                                    consolidate_dataset, how="left", on="employee_code"
)
train.shape

(289, 164)

In [35]:
train[train['employee_code']=='MWM']

,Unnamed: 0_x,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,...,goal_record,sales_customer_base_exp,volunteer_exp,problem_solver,sports_mention,communication_skills,team_player,leadership_mention,average_tenure_per_job,shortest_tenure
288,2745,MWM,2019-04-15,Male,ASSISTANT MANAGER,Permanent Full Time,1988,Millenial: 1983-1999,NaN,NaN,...,1,1,0,0,0,0,1,1,60.5,36.0


In [8]:
list(train.columns)

['Unnamed: 0_x',
 'employee_code',
 'max_hire_date',
 'gender',
 'job_title',
 'worker_category',
 'birth_year',
 'generation',
 'termination_date',
 'termination_reason',
 'termination_type',
 'tenure_at_termination',
 'end_perf_window',
 'tenure_at_end_perf_window',
 'employee_name_x',
 'language_x',
 'resume_found',
 'months_high_perf',
 'months_with_perf',
 'hp_perc',
 'hp_class',
 'perf_found',
 'rehired_',
 'referral_flag',
 'exclusion_code',
 'Unnamed: 0_y',
 'employee_name_y',
 'store',
 'raw_resume',
 'resume_text',
 'resume_bline',
 'language_y',
 'file_type',
 'clean_text',
 'work1_company',
 'work1_length',
 'work2_company',
 'work2_length',
 'work3_company',
 'work3_length',
 'work4_company',
 'work4_length',
 'work5_company',
 'work5_length',
 'work6_company',
 'work6_length',
 'work7_company',
 'avg',
 'job_hopper',
 'work1_flag',
 'work2_flag',
 'work3_flag',
 'work4_flag',
 'work5_flag',
 'work6_flag',
 'work7_flag',
 'competitor_experience',
 'Freedom_competitor_exp',

In [9]:
education_concentration.columns

Index(['Unnamed: 0', 'employee_code', 'employee_name', 'education1_label',
       'education2_label', 'education3_label', 'accounting_concentration',
       'arts_concentration', 'audio_technician_concentration',
       'business_concentration', 'communication_concentration',
       'computer_systems_concentration', 'criminology_concentration',
       'dental_concentration', 'economics_concentration',
       'education_concentration', 'engineering_concentration',
       'english_concentration', 'finance_concentration',
       'general_concentration', 'healthcare_concentration',
       'hospitality_concentration', 'human_resource_concentration',
       'interactive_arts_and_technology_concentration',
       'kinesiology_concentration', 'law_concentration',
       'marketing_concentration', 'not_specified_concentration',
       'other_concentration', 'physics_concentration',
       'psychology_concentration', 'science_concentration',
       'sociology_concentration', 'statistics_concentr

In [10]:
train = train[["employee_code", 'rehired_', 'referral_flag', #train_dataset 
               "job_hopper","average_tenure_per_job","shortest_tenure",          #avg_work 
               'competitor_experience',
               'Freedom_competitor_exp', 'Koodo_competitor_exp', 'Shaw_competitor_exp',
               'Telus_competitor_exp', 'Bell_competitor_exp', 'Rogers_competitor_exp',
               'The Mobile Shop_competitor_exp', 'Best Buy_competitor_exp',
               'Videotron_competitor_exp', 'Wow[!]* Mobile_competitor_exp',
               'The Source_competitor_exp', 'Walmart_competitor_exp',
               'Virgin Mobile_competitor_exp', 'Osl_competitor_exp',    #competitor_experience
               'accounting_concentration',
       'arts_concentration', 'business_concentration',
       'computer_systems_concentration', 'engineering_concentration',
       'finance_concentration', 'general_concentration',
       'human_resource_concentration',
       'interactive_arts_and_technology_concentration',
       'marketing_concentration', 'not_specified_concentration',
       'other_concentration',#education concentration 
               #'administrative', 'assistant manager', 'blue collar', 'cashier', 'cook',
               #'customer service representative', 'driver', 'education',
               #'financial services', 'fitness/sports', 'manager', 'no work', 'other',
               #'sales associate', 'technicians', 'telemarketers', # work_title 
               'highest_degree', 'background_highest_degree', 'country_highest_degree', #degree
               #'sales_exp', 'customer_serv_exp','leader_ship_exp', #sales_cus_exp
               #'efficient service', 'mobile expert', 'high school',
               #'information system', 'cash register', # word count
               'raw_date_chall_readability','clean_Flesch-Kincaid_readability','' # communication level 
               'food_service_industry_exp',
               'apparel_industry_exp', 'supercenter_convenience_industry_exp',
               'automotive_sales_industry_exp', 'blue_collar_industry_exp',
               'consumer_electronics', #industry_exp
               'no_lang_spoken', 'trilingual_flag', 'goal_record',
               'sales_customer_base_exp', 'volunteer_exp', 'problem_solver',
               'sports_mention', 'communication_skills', 'team_player',
               'leadership_mention', # others features in consolidate dataset 
               'clean_text', # resume 
               'hp_class']]

In [11]:
train.referral_flag = train.referral_flag.fillna(0)
train.rehired_ = train.rehired_.fillna("No")

In [12]:
train[train.not_specified_concentration==1][['accounting_concentration',
       'arts_concentration', 'business_concentration',
       'computer_systems_concentration', 'engineering_concentration',
       'finance_concentration', 'general_concentration',
       'human_resource_concentration',
       'interactive_arts_and_technology_concentration',
       'marketing_concentration', 'not_specified_concentration',
       'other_concentration']]



,accounting_concentration,arts_concentration,business_concentration,computer_systems_concentration,engineering_concentration,finance_concentration,general_concentration,human_resource_concentration,interactive_arts_and_technology_concentration,marketing_concentration,not_specified_concentration,other_concentration
0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0,0,1,0,1,0
3,0,0,1,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
284,0,0,0,0,0,0,0,0,0,0,1,0
285,0,0,0,0,1,0,0,0,0,0,1,0
286,0,0,0,0,0,0,0,0,0,0,1,0
287,0,0,0,0,0,0,0,0,0,0,1,0


In [13]:
# regroup country_highest_degree
other_countries = ['dubai', 'pakistan', 'uk', 'china',
       'england', 'syria', 'indore', 'usa', 'philippine', 'vietnam',
       'brazil', 'turkey', 'united kingdom', 'punjab', 'taiwan', 'u']

In [14]:
train

,employee_code,rehired_,referral_flag,job_hopper,average_tenure_per_job,shortest_tenure,competitor_experience,Freedom_competitor_exp,Koodo_competitor_exp,Shaw_competitor_exp,...,goal_record,sales_customer_base_exp,volunteer_exp,problem_solver,sports_mention,communication_skills,team_player,leadership_mention,clean_text,hp_class
0,MZE,No,1.0,0.0,22.00,14.0,1,0,0,0,...,1,1,1,0,0,1,1,1,Avenue Tel reference available request objecti...,0.0
1,MRU,No,0.0,0.0,12.33,3.0,1,0,0,0,...,1,1,1,0,0,1,0,1,Resume obtain utilize opportunity grow benefit...,0.0
2,LAP,Yes,0.0,0.0,40.33,8.0,0,0,0,0,...,1,1,0,1,0,1,1,1,Employment History Hill August present sale Pr...,0.0
3,MYD,No,0.0,0.0,25.17,8.0,0,0,0,0,...,1,1,1,0,0,1,1,0,Manston Cres career objective seek time sale r...,1.0
4,JJB,Yes,0.0,0.0,15.33,0.0,0,0,0,0,...,0,1,0,0,0,1,0,0,Resume Contact LinkedIn Skills Microsoft Offic...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,MTI,No,0.0,0.0,46.75,7.0,1,0,0,0,...,1,1,1,0,0,0,0,1,Jess Road Saint work experience January Januar...,0.0
285,N8M,No,0.0,0.0,38.67,12.0,0,0,0,0,...,1,1,0,0,0,1,1,1,ARVI Thorncliffe Park CAREER OBJECTIVE work re...,0.0
286,MZX,No,0.0,0.0,21.00,4.0,1,0,0,0,...,1,1,0,1,0,0,1,0,PARAST lph Yonge st job OBJECTIVE gain profess...,0.0
287,N4J,No,0.0,1.0,3.00,3.0,0,0,0,0,...,0,1,0,0,0,0,0,0,Resume Sudury Work Experience Companionship Co...,0.0


In [15]:
train.country_highest_degree = train.country_highest_degree.apply(lambda x: "others" if x in other_countries else x)

In [16]:
train.country_highest_degree.value_counts()

canada     178
unknown     56
india       28
others      27
Name: country_highest_degree, dtype: int64

In [17]:
# regroup background_highest_degree
other_backgrounds = ['audio technician', 'audio technician', 'kinesiology', 'blue collar', 'economics', 'sociology', 
                     'kinesiology','physic', 'statistic', 'hospitality', 'criminology', 'english', 'dental', 'human resource',
                    'healthcare', 'communication', 'education']
train.background_highest_degree = train.background_highest_degree.apply(lambda x: "others" if x in other_backgrounds else x)
train.background_highest_degree.unique()

array(['notspecified', 'others', 'interactive art technology', 'business',
       'computer system', 'finance', 'marketing', 'engineering',
       'science', 'general', 'accounting', 'law', 'art', 'psychology',
       nan], dtype=object)

In [18]:
train.background_highest_degree.value_counts()

notspecified                  61
business                      51
others                        41
general                       32
computer system               16
law                           12
engineering                   11
art                           10
finance                        9
science                        9
interactive art technology     8
marketing                      8
accounting                     6
psychology                     5
Name: background_highest_degree, dtype: int64

In [19]:
train.to_csv("../data/0604_training_dataset.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../data/0604_training_dataset.csv'

--- 
# Update industry categories 

In [20]:
new_industry_cat = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/03_feature_extraction/output/job_category_features_2020_06_04_V3.csv")
new_industry_cat = new_industry_cat.fillna("")
new_industry_cat.drop_duplicates("employee_code", inplace=True)

In [21]:
new_industry_cat["industry_expr"] = new_industry_cat.apply(lambda x: list([x['industry_1'],
                                        x['industry_2'],
                                        x['industry_3'], 
                                      x['industry_4'],
                                      x['industry_5'],
                                      x['industry_6'],
                                      x['industry_7']]),axis=1) 

In [22]:
df = new_industry_cat[["industry_expr"]]
df

,industry_expr
0,"[Telecommunications, Telecommunications, , , ,..."
1,"[Telecommunications, Telecommunications, , , ,..."
2,"[Other, Other, , , , , ]"
3,"[Sport_Travel_Enterntain_Hotel, Sport_Travel_E..."
4,"[Sport_Travel_Enterntain_Hotel, Telecommunicat..."
...,...
284,"[unknown, Other, Telecommunications, unknown, ..."
285,"[unknown, Telecommunications, unknown, , , , ]"
286,"[Food-Convenience-Pharmacy, unknown, Food-Conv..."
287,"[Food-Convenience-Pharmacy, unknown, Food-Conv..."


In [23]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

res = pd.DataFrame(mlb.fit_transform(df.industry_expr),
                   columns=mlb.classes_+"_industry_exp",
                   index=df.index)

In [24]:
res

,_industry_exp,Clothing & Footwear_industry_exp,Consumer electronics_industry_exp,Food Service_industry_exp,Food-Convenience-Pharmacy_industry_exp,Other_industry_exp,Sport_Travel_Enterntain_Hotel_industry_exp,Telecommunications_industry_exp,unknown_industry_exp
0,1,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,1,0
2,1,0,0,0,0,1,0,0,0
3,1,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...
284,1,0,0,0,0,1,0,1,1
285,1,0,0,0,0,0,0,1,1
286,1,0,0,0,1,0,0,0,1
287,1,0,0,0,1,0,0,1,1


In [25]:
res.columns

Index(['_industry_exp', 'Clothing & Footwear_industry_exp',
       'Consumer electronics_industry_exp', 'Food Service_industry_exp',
       'Food-Convenience-Pharmacy_industry_exp', 'Other_industry_exp',
       'Sport_Travel_Enterntain_Hotel_industry_exp',
       'Telecommunications_industry_exp', 'unknown_industry_exp'],
      dtype='object')

In [29]:
print(train.shape)
print(res.shape)

(289, 56)
(288, 9)


In [30]:
train_new = pd.concat([train, res], axis=1)

In [31]:
train_new

,employee_code,rehired_,referral_flag,job_hopper,average_tenure_per_job,shortest_tenure,competitor_experience,Freedom_competitor_exp,Koodo_competitor_exp,Shaw_competitor_exp,...,hp_class,_industry_exp,Clothing & Footwear_industry_exp,Consumer electronics_industry_exp,Food Service_industry_exp,Food-Convenience-Pharmacy_industry_exp,Other_industry_exp,Sport_Travel_Enterntain_Hotel_industry_exp,Telecommunications_industry_exp,unknown_industry_exp
0,MZE,No,1.0,0.0,22.00,14.0,1,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,MRU,No,0.0,0.0,12.33,3.0,1,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,LAP,Yes,0.0,0.0,40.33,8.0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,MYD,No,0.0,0.0,25.17,8.0,0,0,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,JJB,Yes,0.0,0.0,15.33,0.0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,MTI,No,0.0,0.0,46.75,7.0,1,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
285,N8M,No,0.0,0.0,38.67,12.0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
286,MZX,No,0.0,0.0,21.00,4.0,1,0,0,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
287,N4J,No,0.0,1.0,3.00,3.0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


In [39]:
train_new[train_new['employee_code']=='MWM']

,employee_code,rehired_,referral_flag,job_hopper,average_tenure_per_job,shortest_tenure,competitor_experience,Freedom_competitor_exp,Koodo_competitor_exp,Shaw_competitor_exp,...,hp_class,_industry_exp,Clothing & Footwear_industry_exp,Consumer electronics_industry_exp,Food Service_industry_exp,Food-Convenience-Pharmacy_industry_exp,Other_industry_exp,Sport_Travel_Enterntain_Hotel_industry_exp,Telecommunications_industry_exp,unknown_industry_exp
288,MWM,No,0.0,0.0,60.5,36.0,0,0,0,0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [40]:
train_new = pd.concat([train, res], axis=1)

In [41]:
train_new = train_new[:288]

In [42]:
train_new[train_new['employee_code']=='MWM']

,Unnamed: 0_x,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,...,shortest_tenure,_industry_exp,Clothing & Footwear_industry_exp,Consumer electronics_industry_exp,Food Service_industry_exp,Food-Convenience-Pharmacy_industry_exp,Other_industry_exp,Sport_Travel_Enterntain_Hotel_industry_exp,Telecommunications_industry_exp,unknown_industry_exp


In [135]:
train_new

,employee_code,rehired_,referral_flag,job_hopper,average_tenure_per_job,shortest_tenure,competitor_experience,Freedom_competitor_exp,Koodo_competitor_exp,Shaw_competitor_exp,...,hp_class,_industry_exp,Clothing & Footwear_industry_exp,Consumer electronics_industry_exp,Food Service_industry_exp,Food-Convenience-Pharmacy_industry_exp,Other_industry_exp,Sport_Travel_Enterntain_Hotel_industry_exp,Telecommunications_industry_exp,unknown_industry_exp
0,MZE,No,1.0,0.0,22.00,14.0,1,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,MRU,No,0.0,0.0,12.33,3.0,1,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,LAP,Yes,0.0,0.0,40.33,8.0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,MYD,No,0.0,0.0,25.17,8.0,0,0,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,JJB,Yes,0.0,0.0,15.33,0.0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,N1Q,No,0.0,1.0,11.50,2.0,1,0,0,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,MTI,No,0.0,0.0,46.75,7.0,1,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
285,N8M,No,0.0,0.0,38.67,12.0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
286,MZX,No,0.0,0.0,21.00,4.0,1,0,0,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [136]:
train_new.to_csv("C:/Users/NIW/Documents/Capstone Project/data/clean_data/0607_training_dataset.csv", index=False)

### Additional Feats

In [137]:
add_feats_01 = pd.read_csv("C:/Users/NIW/Documents/Capstone Project/data/clean_data/additional_set_feats_01.csv")
add_feats_01.drop_duplicates("employee_code", inplace=True)
add_feats_01.head(5)

,employee_code,telco_electro_jobs,telco_electro_recency,recency_type_telco_electro_exp,no_job_categorical,telco_electro_perc_group,read_score_categorical,flag_hd_bachelor_plus,flag_hd_highschool,business_flag,no_jobs
0,N3O,2,1,high,multiple,significant,high,1,0,0,5
1,BDP,2,1,high,few,significant,high,1,0,1,2
2,MZV,0,0,inexistant,multiple,inexistant,high,0,0,0,3
3,N62,1,1,high,multiple,low,high,1,0,0,6
4,N7R,1,1,high,multiple,significant,low,1,0,0,4


In [138]:
train_new = pd.merge(train_new,add_feats_01,on="employee_code",how="inner")
train_new.to_csv("C:/Users/NIW/Documents/Capstone Project/data/clean_data/0607_02_training_dataset.csv", index=False)